In [1]:
# !pip install langchain==0.0.346
# !pip install newspaper3k==0.2.8
# !pip install pandas==2.1.3
# !pip install deltalake==0.14.0
# !pip install tabulate==0.9.0
# !pip install googlenews
# !pip install gnews==0.3.6
# !pip install plotly==5.18.0
# ! pip install nbformat==5.9.2
# !pip install openai==1.3.7
# !pip show 

In [2]:
import sys
sys.path.append('..')

import os
import json
import pandas as pd

from tabulate import tabulate
from IPython.display import display, Markdown
# TEST

from datetime import datetime, timedelta
import pytz

from deltalake import DeltaTable
from deltalake.writer import write_deltalake

from openai import OpenAI

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [3]:
# Params
table_path = '../data/articles-ingest'

# --------------------------------------------
query = "Ozempic Eli Lilly"

days_lookback = 30*2
max_results = 1000
current_datetime = datetime.now().astimezone(pytz.utc)
current_date = current_datetime.date()
start_date = (current_datetime - timedelta(days=days_lookback)).date()

# --------------------------------------------

start_date_str = str(start_date.strftime("%Y-%m-%d"))
current_date_str = str(current_date.strftime("%Y-%m-%d"))

print(
    f"""
    end_date: {current_date}
    start_date: {start_date}
    
    
    days_lookback: {days_lookback}
    max_results: {max_results}
    """
)

print("Harambe was here")


    end_date: 2023-12-07
    start_date: 2023-10-08
    
    
    days_lookback: 60
    max_results: 1000
    
Harambe was here


In [4]:
# Query queries that have been ingested
DeltaTable(table_path).to_pandas(columns=['query']).value_counts()

query            
Return to Office     97
Ozempic              89
Ozempic Eli Lilly    84
Name: count, dtype: int64

In [5]:
df = DeltaTable(table_path).to_pandas(
    filters=[
        ('query','=',query),
        ('published date str','>', start_date_str),
    ]
)

df = df.loc[df['article text'].notna()] # can remove articles where scraping failed

df.head(2)

,title,description,published date,url,publisher,query,query end date,published datetime,published date str,article text,article html,article summary,article keywords
0,Eli Lilly Just Got One Step Closer to Dethroni...,Eli Lilly Just Got One Step Closer to Dethroni...,2023-11-19,https://news.google.com/rss/articles/CBMiXWh0d...,"{'href': 'https://www.fool.com', 'title': 'The...",Ozempic Eli Lilly,2023-12-07 02:38:25.798191,2023-11-19 08:00:00,2023-11-19,Just about every evening when I sit down for d...,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",Let's break down what Eli Lilly has in its pip...,"[step, eli, closer, drug, lilly, ozempic, buy,..."
1,Ozempic Sales Are Skyrocketing Faster Than Exp...,Ozempic Sales Are Skyrocketing Faster Than Exp...,2023-10-17,https://news.google.com/rss/articles/CBMiXWh0d...,"{'href': 'https://www.fool.com', 'title': 'The...",Ozempic Eli Lilly,2023-12-07 02:38:25.798191,2023-10-17 07:00:00,2023-10-17,Everything seems to be going right for Novo No...,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",Ozempic sales are skyrocketing faster than exp...,"[expected, drug, skyrocketing, lilly, ozempic,..."


## process for prompting openai with article info, with traceability

- Each row in the table is an article. Functions can be used to iterate through the table and add additional column.
- For each prompt, we want to retain the prompt in the table, as well as the message history, and response.


In [6]:
client = OpenAI()

def print_df(df):
    return tabulate(df, headers='keys', tablefmt='html')

def generate_openai_response_json(prompt, model="gpt-3.5-turbo-1106"):
    """
    Generic function to generate a GPT response
    """
    response = client.chat.completions.create(
    model=model,
    response_format={ "type": "json_object" },
    messages=[
        {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
        {"role": "user", "content": prompt}
    ]
    )
    json_response = response.choices[0].message.content
    return json.loads(json_response)



def generate_sentiment_prompt(query, title, article_text, published_date_str):
    prompt = f"""
    # You are a reviewer of articles. 
    - Your task is to read the article below, provide a sentiment negativity score.
    - 0 is very positive, 10 is very negative.
    - Please provide short bullet points, each representing specific negative claims made in the article (if any).

    ## Inputs
    - Search term used to find this article: {query}

    - The article:
        - Title: {title}
        - Published Date: {published_date_str}
        - Article: \n{article_text}

    ## Outputs
    Please return the output as a JSON object with the following: sentiment_score (int in string format), sentiment_explanation (string), and negative_claims (string with bullet points delimited by new lines), 
    """
    return prompt

def generate_sentiment_prompt_and_response(query, title, article_text, published_date_str):
    
    sentiment_prompt = generate_sentiment_prompt(query, title, article_text, published_date_str)

    response = generate_openai_response_json(sentiment_prompt)
    return response['sentiment_score'], response['sentiment_explanation'], response['negative_claims']

In [7]:
i=2
article_text = df.iloc[i]['article text']
published_date_str = df.iloc[i]['published date str']
title = df.iloc[i]['title']
query = df.iloc[i]['query']

In [8]:
sentiment_prompt = generate_sentiment_prompt(query, title, article_text, published_date_str)
display(Markdown(sentiment_prompt))


    # You are a reviewer of articles. 
    - Your task is to read the article below, provide a sentiment negativity score.
    - 0 is very positive, 10 is very negative.
    - Please provide short bullet points, each representing specific negative claims made in the article (if any).

    ## Inputs
    - Search term used to find this article: Ozempic Eli Lilly

    - The article:
        - Title: Will Ozempic-type obesity drugs pay off for society? Makers say yes - STAT
        - Published Date: 2023-10-17
        - Article: 
Our expensive obesity drugs are worth it, Novo Nordisk and Eli Lilly argue in raft of studies

DALLAS — Here at ObesityWeek, one of the largest conferences on obesity, Novo Nordisk and Eli Lilly are displaying more than a dozen studies that together carry the message: Our blockbuster weight loss treatments will be worth it for society.

But experts point out that much of this company-funded research does not include the cost of the drugs themselves, which sell at more than $10,000 per year in the U.S. and are meant to be taken indefinitely.

advertisement

The two major pharmaceutical companies have seen interest skyrocket for their GLP-1-based drugs like Ozempic, Wegovy, and Mounjaro, a new generation of medications that can cut substantial amounts of weight. But insurance plans have been slow to grant wide access to obesity treatments, concerned that the vast amount of people eligible to take them, along with their high price, could lead to significant financial strain.

    ## Outputs
    Please return the output as a JSON object with the following: sentiment_score (int in string format), sentiment_explanation (string), and negative_claims (string with bullet points delimited by new lines), 
    

In [9]:
sentiment_score, sentiment_explanation, negative_claims = generate_sentiment_prompt_and_response(query, title, article_text, published_date_str)

print(f"sentiment_score: {sentiment_score}")
print(f"sentiment_explanation: {sentiment_explanation}")
print(f"negative_claims: {negative_claims}")

sentiment_score: 7
sentiment_explanation: The sentiment negativity score is 7, indicating a fairly negative sentiment towards the cost and access to obesity drugs.
negative_claims: - Company-funded research does not include the cost of the drugs themselves
- High price of the drugs, selling at more than $10,000 per year in the U.S.
- Insurance plans have been slow to grant wide access to obesity treatments due to concerns over significant financial strain


In [10]:
df_results = df.apply(lambda row: generate_sentiment_prompt_and_response(row['query'], row['title'], row['article text'], row['published date str']), axis=1, result_type='expand')
df_results.columns = ['sentiment_score', 'sentiment_explanation', 'negative_claims']

df_results['sentiment_score'] = df_results['sentiment_score'].astype(int)

df = pd.concat([df,df_results], axis=1)

In [14]:
df = df.sort_values(by=['sentiment_score'], ascending=False)

In [15]:
df.head(5)

,title,description,published date,url,publisher,query,query end date,published datetime,published date str,article text,article html,article summary,article keywords,sentiment_score,sentiment_explanation,negative_claims
34,"Pfizer struggles to catch up to Ozempic, Wegov...","Pfizer struggles to catch up to Ozempic, Wegov...",2023-12-02,https://news.google.com/rss/articles/CBMidmh0d...,"{'href': 'https://www.latimes.com', 'title': '...",Ozempic Eli Lilly,2023-12-07 02:38:25.798191,2023-12-02 11:00:39,2023-12-02,"On Friday, Pfizer announced that it’s dropping...","<!DOCTYPE html><html class=""page article-page""...","On Friday, Pfizer announced that it’s dropping...","[pills, catch, ozempic, struggles, experienced...",8,The article portrays a negative sentiment towa...,- Pfizer is dropping development of an experim...
40,Woman dies after taking Ozempic to slim down f...,Woman dies after taking Ozempic to slim down f...,2023-11-06,https://news.google.com/rss/articles/CBMiYWh0d...,"{'href': 'https://nypost.com', 'title': 'New Y...",Ozempic Eli Lilly,2023-12-07 02:38:25.798191,2023-11-06 08:00:00,2023-11-06,An Australian woman who took Ozempic to shed a...,"<!doctype html>\n<html lang=""en-US"">\n\n<head ...",Intestinal blockage is called “ileus” — the FD...,"[daughters, drug, shouldnt, woman, minutes, 60...",8,The sentiment negativity score for the article...,- Use of Ozempic led to the death of a woman\n...
36,Anti-obesity drugs Ozempic and Mounjaro can sh...,Anti-obesity drugs Ozempic and Mounjaro can sh...,2023-10-24,https://news.google.com/rss/articles/CBMihQFod...,"{'href': 'https://english.elpais.com', 'title'...",Ozempic Eli Lilly,2023-12-07 02:38:25.798191,2023-10-24 07:00:00,2023-10-24,Starry-eyed venture capitalists love to talk a...,"<!DOCTYPE html><html lang=""en-us""><head><scrip...",New anti-obesity drugs have the potential to t...,"[medical, antiobesity, ozempic, mounjaro, reve...",8,The article discusses potential negative impac...,- New anti-obesity drugs have the potential to...
15,Novo Nordisk to reduce supply of Victoza to ma...,Novo Nordisk to reduce supply of Victoza to ma...,2023-11-22,https://news.google.com/rss/articles/CBMicmh0d...,"{'href': 'https://www.fiercepharma.com', 'titl...",Ozempic Eli Lilly,2023-12-07 02:38:25.798191,2023-11-22 08:00:00,2023-11-22,Unable to scale up its manufacturing fast enou...,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" pre...",Unable to scale up its manufacturing fast enou...,"[shortage, drug, production, boost, quarter, o...",8,The article contains several negative claims a...,Novo Nordisk facing shortage of Victoza\nReduc...
33,Pfizer's stock falls 6% after halting trial of...,Pfizer's stock falls 6% after halting trial of...,2023-12-01,https://news.google.com/rss/articles/CBMihgFod...,"{'href': 'https://www.morningstar.com', 'title...",Ozempic Eli Lilly,2023-12-07 02:38:25.798191,2023-12-01 15:20:00,2023-12-01,By Ciara Linnane\n\nPfizer says it will not mo...,"<!doctype html>\n<html data-n-head-ssr lang=""e...",The stock was on track for its biggest one-day...,"[drug, dow, danuglipron, jones, eli, halting, ...",7,The sentiment negativity score for the article...,- Pfizer's stock tumbled 6%\n- Pfizer's stock ...


In [24]:
print_df(df.head(5).drop(columns=['article text','article html']))

,title,description,published date,url,publisher,query,query end date,published datetime,published date str,article summary,article keywords,sentiment_score,sentiment_explanation,negative_claims
34,"Pfizer struggles to catch up to Ozempic, Wegovy with weight-loss pill - Los Angeles Times","Pfizer struggles to catch up to Ozempic, Wegovy with weight-loss pill Los Angeles Times",2023-12-02,https://news.google.com/rss/articles/CBMidmh0dHBzOi8vd3d3LmxhdGltZXMuY29tL2J1c2luZXNzL3N0b3J5LzIwMjMtMTItMDIvcGZpemVyLXN0cnVnZ2xlcy10by1jYXRjaC11cC10by1vemVtcGljLXdlZ292eS13aXRoLXdlaWdodC1sb3NzLXBpbGzSAQA?oc=5&hl=en-US&gl=US&ceid=US:en,"{'href': 'https://www.latimes.com', 'title': 'Los Angeles Times'}",Ozempic Eli Lilly,2023-12-07 02:38:25.798191,2023-12-02 11:00:39,2023-12-02,"On Friday, Pfizer announced that it’s dropping development of an experimental weight-loss pill after patients experienced uncomfortable side effects in clinical trials. A pill has been widely viewed as a way for new players to make inroads in a frenzied new weight-loss market dominated by the makers of injectable Ozempic and Mounjaro. Some drugs earlier in development rival a level of weight loss only thought possible with bariatric surgery. A pill might persuade more patients to take weight-loss drugs. Rybelsus, Novo’s pill version of the diabetes shot Ozempic that’s often used for weight loss, is available at $936 for a one-month supply.",['pills' 'catch' 'ozempic' 'struggles' 'experienced' 'pfizer' 'loss' 'pill' 'drugs' 'patients' 'weight' 'wegovy' 'rival' 'weightloss'],8,"The article portrays a negative sentiment towards Pfizer's struggle to develop an effective weight-loss pill, highlighting the challenges, side effects, and limitations compared to existing injectable drugs.","- Pfizer is dropping development of an experimental weight-loss pill due to uncomfortable side effects in clinical trials. - Pfizer's weight-loss pill, danuglipron, resulted in high rates of nausea, vomiting, and diarrhea in patients, making it unsuitable for twice-daily use. - Pills like Pfizer's small molecule drugs may pose a higher risk for side effects, including the risk of liver toxicity. - Pfizer's pill development is far behind in comparison to the existing injectable weight-loss drugs, struggling to match their efficacy and formulation."
40,Woman dies after taking Ozempic to slim down for daughter's wedding: 'She shouldn't be gone' - New York Post,Woman dies after taking Ozempic to slim down for daughter's wedding: 'She shouldn't be gone' New York Post,2023-11-06,https://news.google.com/rss/articles/CBMiYWh0dHBzOi8vbnlwb3N0LmNvbS8yMDIzLzExLzA2L2xpZmVzdHlsZS93b21hbi1kaWVzLWFmdGVyLXRha2luZy1vemVtcGljLXRvLXNsaW0tZG93bi1mb3Itd2VkZGluZy_SAWVodHRwczovL255cG9zdC5jb20vMjAyMy8xMS8wNi9saWZlc3R5bGUvd29tYW4tZGllcy1hZnRlci10YWtpbmctb3plbXBpYy10by1zbGltLWRvd24tZm9yLXdlZGRpbmcvYW1wLw?oc=5&hl=en-US&gl=US&ceid=US:en,"{'href': 'https://nypost.com', 'title': 'New York Post'}",Ozempic Eli Lilly,2023-12-07 02:38:25.798191,2023-11-06 08:00:00,2023-11-06,"Intestinal blockage is called “ileus” — the FDA received 18 reports of it in people taking Ozempic as of late September. 60 Minutes Australia“She had a little bit of brown stuff coming out of her mouth, and I realized she wasn’t breathing and started doing CPR,” Roy Webster recounted to “60 Minutes Australia” last week. 6 On Jan. 16, just a few months before her daughter’s wedding, Webster’s husband allegedly found her unconscious with a brown liquid seeping out of her mouth. 60 Minutes Australia“If I knew that could happen, she wouldn’t have been taking it,” the grieving husband insisted. 60 Minutes AustraliaGastroparesis is just one of many side effects that have been reported, including “Ozempic finger,” “Ozempic burp,” “Ozempic butt,” “Ozempic face” and weird dreams about celebrities.",['daughters' 'drug' 'shouldnt' 'woman' 'minutes' '60' 'ozempic' 'dies' 'webster' 'worth' 'husband' 'gastrointestinal' 'taking' 'slim' 'gone' 'wedding'],8,"The sentiment